We want to experiment with BVP and RMC for this we need green functions see period2
we will be using the green functions of $y''$
## boundary green function of $y''$

$$
\begin{align*}
G''_{b_{0}}(t) &= 0, G(b_{0})=1,G(b_{1})=0 \Rightarrow \\
G_{b_{0}}(t) &= \frac{b_{1}-t}{b_{1}-b_{0}} \\   
\end{align*}
$$ 
$$
\begin{align*}
G''_{b_{1}}(t) &= 0, G(b_{0})=0,G(b_{1})=1 \Rightarrow \\
G_{b1}(t) &= \frac{t-b_{0}}{b_{1} -b_{0}}
\end{align*}
$$


## source green function of $y''$
$$
\begin{align*}
G''_{s}(t) &= \delta(t-s), G(b_{0})=0,G(b_{1})=0 \Rightarrow \\
G_{s}(t) &=    \begin{cases}
        -\frac{(b_{1}-s)(t-b_{0})}{b_{1}-b_{0}} & \text{if } t<s\\
        -\frac{(b_{1}-t)(s-b_{0})}{b_{1}-b_{0}} & \text{if } s<t
    \end{cases}
\end{align*}
$$ 

## tests of the green function
We solve
$$
y''= y,y(b_{0}), y(b_{1}).
$$ 
to test these green functions are correct



In [382]:
from random import random
from math import exp

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = random()
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)
    
def soltest(t,y0,y1,b0,b1,nsim): return sum(Y(t,y0,y1,b0,b1) for _ in range(nsim))/nsim

t,nsim = 0.5,10**4
print((soltest(t,1,exp(2),0,2,nsim)-exp(t))/exp(t))

0.0046536961596754635


some old text (in the comments)
<!-- Lets now do a linear boundary problem:
$$
y''= y, y(-1)=e^{-1},y(1)=e^{1}.
$$ 
with somekind of RRMC, coupled recursion and recursion on the boundary. To turn this into an integral equation we will use green functions. We will use the green function of $y''$ on $[0,1]$ because we can because we can abuse symmetries (this is a special case of walk on spheres).

We will do 2-phase coupled recursion with $y(-0.5),y(0.5)$ and $y(0)$ by integrating over $[-0.5,0.5]$ and $[-1,0],[0,1]$ but lets first test if the translation symmetry works by solving on these domains with exact boundaries. -->

Our main example will be solving 
$$
y''=y, y(-1) = e^{-1},y(1)=e .
$$
on equally spaced grid so we can test convergence stuff. But first will do a subcase of this a grid with $3$ points $[-0.5,0,0.5]$. <br> 

We will implement this with recursion in recursion. The outer recursion is a coupled recursion of $3$ points and the inner recursion is just regular RMC where we assume we know the value at adjacent points.

In [402]:
from random import random
from math import exp
import numpy as np

def Gb0(t,b0,b1): return (b1-t)/(b1-b0)
def Gb1(t,b0,b1): return (t-b0)/(b1-b0)
def G(t,s,b0,b1): return -(b1-s)*(t-b0)/(b1-b0) if t<s else - (b1-t)*(s-b0)/(b1-b0) 

def Y(t,y0,y1,b0,b1): 
    l = 2 # russian roulette rate
    if random()*l>1: return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1
    S = random()
    return Gb0(t,b0,b1)*y0 + Gb1(t,b0,b1)*y1 + l*G(t,S,b0,b1) * Y(S,y0,y1,b0,b1)*(b1-b0)

def X_slow():
    l = 2 # russian roulette rate
    if random()*l>1: return  np.zeros(3)
    X = X_slow()
    x0 = soltest(t,exp(-1),X[1],-1,0,1)
    x1 = soltest(t,X[0],X[2],-0.5,0.5,1)
    x2 = soltest(t,X[1],exp(1),0,1,1)
    """     x0 = Y(t,exp(-1),X[1],-1,0)
    x1 = Y(t,X[0],X[2],-0.5,0.5)
    x2 = Y(t,X[1],exp(1),0,1) """
    return l*np.array([x0,x1,x2])

def soltest2(nsim): return sum(X_slow() for _ in range(nsim))/nsim
nsim = 10**3
print(soltest2(nsim))

[13.04397704  2.98590874  2.1999369 ]
